In [1]:
import pandas as pd
import os
import numpy as np
import re
from datetime import datetime
from pymongo import MongoClient

In [211]:
arr = os.listdir('./excel')
file = [f for f in arr if f[-3:]=='xls']
f=file[0]
### Loading sheets into DF ###
df = pd.read_excel('./excel/'+f,skiprows=[5,9,1], sheet_name=[0,1,2],header=None)
### Formatting ###
t = df[0].loc[3,[1,2]].str.extract('(\d*\s)',expand=False).astype(int)

### Formatting strings ###
for x in range(3):
    df[x].loc[3,[1,2]] = df[x].loc[3,[1,2]].str.extract('(\d*\s)',expand=False).astype(int)
    df[x].loc[6] = df[x].loc[6].str.extract('(\d*\.\d+|\d+)', expand=False).astype(float)  

### Extracting numbers from str ###
    for i in range(9,len(df[x]),2):
        df[x].loc[i] = df[x].loc[i].str.extract('(\d*\.\d+|\d+)', expand=False).astype(float) 
### Conversion from str to numeric ###    
#     l = [2,5,6]
#     for i in l:
#         df[x].iloc[i,1:] = pd.to_numeric(df[x].iloc[i,1:], errors = 'coerce',downcast='float')
# #         print(df[x].iloc[i,1:])
#     for i in range(8,len(df[x].index)):
#         df[x].iloc[i,1:8] = pd.to_numeric(df[x].iloc[i,1:8], errors = 'coerce',downcast='float')
#         if(x==0):
#             df[x].iloc[i,8] = pd.to_numeric(df[x].iloc[i,8], errors = 'coerce',downcast='float')      
i =0
bytype = df[i].iloc[8:].dropna(axis = 1, how = 'all')
col_index = df[i].iloc[7,:].dropna()
bytype.columns = col_index
bytype.set_index(df[i].iloc[7,0],inplace = True)
bytype.index = pd.Series(bytype.index).fillna(method='ffill')
# byType = bytype.groupby(df[i].iloc[7,0]).agg(lambda x: list(x))
# byType.columns = df[i].iloc[7,1:].dropna()
# byType.reset_index(inplace=True)
# byType.rename(columns={byType.columns[0]:'area'},inplace=True)
# resp = byType.to_dict('record')

bytype = bytype.reset_index()
for i in range(1,len(bytype.index),2):
    bytype.iloc[i,0] = bytype.iloc[i,0] + "_pct"
bytype.rename(columns={bytype.columns[0]:'area'},inplace=True)
by = bytype.to_dict('record')
# bytype
head = df[0].iloc[4,1:].dropna()
allResponses = df[0].iloc[5:7,1:].dropna(axis = 1, how = 'all')
allResponses.columns = head
allResponses = allResponses.set_index(pd.Index(["resp_no","resp_pct"]))
allResponses = allResponses.reset_index()
allResponses.rename(columns={allResponses.columns[0]:'resp'},inplace=True)
all_resp = allResponses.to_dict('record')

res = cleanNull(by)
res
# respon = []
# for d in by:
#     clean={}
#     for k,v in d.items():
#         if not(pd.isnull(v)):
#             clean[k] = v
#         else:
#             pass
#     respon.append(clean)


[{'area': 'Area 1',
  'Excellent': 2,
  'Good': 0,
  'Average': 0,
  'Poor': 0,
  'Bad': 0,
  'Total Responses': 2,
  'Exp. Index': 95.0,
  'Improvement Index': 90.0},
 {'area': 'Area 1_pct',
  'Excellent': 100.0,
  'Good': 0.0,
  'Average': 0.0,
  'Poor': 0.0,
  'Bad': 0.0},
 {'area': 'Area 4',
  'Excellent': 96,
  'Good': 14,
  'Average': 10,
  'Poor': 9,
  'Bad': 13,
  'Total Responses': 142,
  'Exp. Index': 77.08,
  'Improvement Index': -7.21},
 {'area': 'Area 4_pct',
  'Excellent': 67.61,
  'Good': 9.86,
  'Average': 7.04,
  'Poor': 6.34,
  'Bad': 9.15},
 {'area': 'Area 2',
  'Excellent': 101,
  'Good': 228,
  'Average': 6,
  'Poor': 1,
  'Bad': 13,
  'Total Responses': 349,
  'Exp. Index': 75.66,
  'Improvement Index': 0.07},
 {'area': 'Area 2_pct',
  'Excellent': 28.94,
  'Good': 65.33,
  'Average': 1.72,
  'Poor': 0.29,
  'Bad': 3.72},
 {'area': 'Area 5',
  'Excellent': 31,
  'Good': 9,
  'Average': 4,
  'Poor': 5,
  'Bad': 10,
  'Total Responses': 59,
  'Exp. Index': 67.51,
  

In [209]:
def cleanNull(data):
#     cleanData = []
#     for d in data:
#         clean = {}   
#         for k, v in d.items():
#             if isinstance(v, dict):
#                 nested = cleanNull(v)
#                 if len(nested.keys()) > 0:
#                     clean[k] = nested
#             elif v is not None:
#                 if isinstance(v,list):
#                     v = [v for v in v if not(pd.isnull(v))]
#                     clean[k] = v
#                 elif not(pd.isnull(v)):
#                     clean[k] = v
#                 else:
#                     pass  
#             cleanData.append(clean)
#     return cleanData    
 
#     cleanData = []
#     for d in by:
#         clean={}
#         for k,v in d.items():
#             if not(pd.isnull(v)):
#                 clean[k] = v
#             else:
#                 pass
#         cleanData.append(clean)
#     return cleanData

    cleanData = []
    for d in data:
        clean={}
        for k,v in d.items():
            if not(pd.isnull(v)):
                if isinstance(v,str):
                    if re.match('(-*\d+.*\d*)',v):
                        v = float(v)
                clean[k] = v
            else:
                pass
        cleanData.append(clean)
    return cleanData

In [219]:
# if (isinstance(res[0]["Improvement Index"],str) and re.match('\d*',res[0]["Exp. Index"]))
# res[0]["Improvement Index"] = float(res[0]["Improvement Index"])
# res[0]["Improvement Index"]
z = re.match('(\d*.*\d*)',res[0]["area"])
res[0]["Exp. Index"]
if re.match('-*[0-9]+([.][0-9]+)?$',str(res[1]["area"])):
    print("bullshit")
print("END")

END
